In [1]:
import torch
from transformers import WhisperForConditionalGeneration, WhisperProcessor

torch.cuda.empty_cache()

device = "cuda" if torch.cuda.is_available() else "cpu"

model = WhisperForConditionalGeneration.from_pretrained("checkpoint\checkpoint-5000").to(device)
processor = WhisperProcessor.from_pretrained("openai/whisper-large-v2", language="hi", task="transcribe")



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
import librosa
import torch


def transcribe(loc):
    # load model and processor
    y, s = librosa.load(loc, sr=16000)
    
    waveform = y
    sampling_rate = s
    
    #print(waveform)

    input_features = processor(waveform, sampling_rate=sampling_rate, return_tensors="pt").input_features
    input_features = input_features.cuda()

    predicted_ids = model.generate(input_features)

    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)

    return(transcription[0])

In [3]:
import os

aud_list = os.listdir(r"D:/Clg/MTP/dataset-mundari-tts/all_audio/")



In [4]:
import random
sampled_list = []
for x in range(0,100):
    sampled_list.append(random.choice(aud_list))


In [5]:
from tqdm import tqdm
test = {}
for x in tqdm(sampled_list):
    t = transcribe("D:/Clg/MTP/dataset-mundari-tts/all_audio/"+x)
    test[x] = t

100%|██████████| 100/100 [1:02:48<00:00, 37.68s/it]


In [8]:
test

{'female_7258.wav': 'खोबोर साकाम रआः खुरजि विज्ञापन रे जुड़ागाकाना।',
 'female_3618.wav': 'बागेश्वर रे जानाव होड़ोको गोए जाः कोआ एनाते होड़ोको खैरवाकाना।',
 'female_12753.wav': 'नगेन बोंगा बुरू ओड़ोः जादु टोना रिका केदाए।',
 'female_8049.wav': 'गपा वु नपामा एटाः ओनोल को लोः।',
 'female_18074.wav': 'विकिलीक्स रेन संस्थापक जुलियन असांजे लन्दन रे गिरफ्तार कि://आ।',
 'female_12171.wav': 'इमताङ खींसते सुमति आञ साला काजि केआते एराङ जाञ ताइकेना।',
 'female_4679.wav': 'काको लेलेआचि नि पापतादाए चि पुण्य।',
 'female_8131.wav': 'जांए होड़ोकोरेओ नेका शक्तिको ताइना।',
 'male_1506.wav': 'बरना गी नेया ते अदिका मेना',
 'female_833.wav': 'इनिःअः रअःअयुम केद् बाहिर रे होड़ोको हुंडियना।',
 'female_12505.wav': 'जदि ओफिस मेना रेदो टेबल अर कुर्सि को आलोपे जोगाव या।',
 'female_19314.wav': 'भारतीय दर्शनकोका ऐतिहासिक क्रम निश्चितरिका बालाएगेआ।',
 'female_2705.wav': 'बोगोमान लेलकेदाए चि गोटा संसार इपा गे मेना।',
 'female_16300.wav': 'मुद परेश बरूआ लोःते नापाम लेना।',
 'female_13978.wav': 'एराको  अर सेपेड़ेद्-हपानु

In [12]:
gold = {}
for x in tqdm(sampled_list):
    x = x[:-4]
    #print(x)
    t = open("D:/Clg/MTP/dataset-mundari-tts/data-full/all_transcipt/"+x+'.txt', "r",encoding="utf8")
    tr = t.read()
    gold[x+'.wav'] = tr
    

100%|██████████| 100/100 [00:00<00:00, 7064.33it/s]


In [13]:
import pandas as pd
data = pd.DataFrame(columns=['Audio', 'Gold','Transcription'])

for x in gold.keys():
    print(x)
    print(gold[x])
    print(test[x])
    data = data._append({'Audio': x, 'Gold': gold[x],'Transcription':test[x]}, ignore_index=True)
    

female_7258.wav
खोबोर साकाम रआः खुरजि विज्ञापन रे जुड़ागाकाना।

खोबोर साकाम रआः खुरजि विज्ञापन रे जुड़ागाकाना।
female_3618.wav
बागेश्वर रे जानाव होड़ोको गोए: जा: कोआ एनाते होड़ोको खैरवाकना।

बागेश्वर रे जानाव होड़ोको गोए जाः कोआ एनाते होड़ोको खैरवाकाना।
female_12753.wav
नगेन बोंगा बुरू ओड़ो: जादु टोना रिका केदाए।

नगेन बोंगा बुरू ओड़ोः जादु टोना रिका केदाए।
female_8049.wav
गपा वु नपामा एटाः ओनोल को लोः।

गपा वु नपामा एटाः ओनोल को लोः।
female_18074.wav
विकिलीक्स रेन संस्थापक जुलियन असांजे लन्दन रे गिरफ्तार कि:आ।

विकिलीक्स रेन संस्थापक जुलियन असांजे लन्दन रे गिरफ्तार कि://आ।
female_12171.wav
इमताङ खींसते सुमति आञ साला काजि केआते एराङ जाञ ताइकेना।

इमताङ खींसते सुमति आञ साला काजि केआते एराङ जाञ ताइकेना।
female_4679.wav
काको लेलेआचि नि पापतादाए चि पुण्य।

काको लेलेआचि नि पापतादाए चि पुण्य।
female_8131.wav
जांए होड़ोकोरेओ नेका शक्तिको ताइना।

जांए होड़ोकोरेओ नेका शक्तिको ताइना।
male_1506.wav
बरना: गी नेया ते अदिका मेना:

बरना गी नेया ते अदिका मेना
female_833.wav
इनिःअः रअःअयुम केद् बाहिर रे 

In [15]:
data.to_excel('run3_results.xlsx')